This time we are going to use this dataset to continue with our practicing. As I said in Part 1, you can download the book here:

http://www3.canisius.edu/~yany/python/Python4DataAnalysis.pdf


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
Nnames = pd.read_csv('../input/NationalNames.csv')

Nnames.head()

We start using the function 'pivot_table' to mold the data. We want to check how many boys and girls were named each year.

In [ ]:
Nnames_sex = Nnames.pivot_table('Count', index = 'Year', columns = 'Gender', aggfunc = sum)
Nnames_sex.head()

In [ ]:
One advantage of 'pivot_table' is that we can plot it easily.

In [ ]:
Nnames_sex.plot(title ="Total births by sex and year");

We create a function 'add_prop' in order to create a new column, 'prop', which will tell the percentage of  males and females that have been given each name every year.

In [ ]:
def add_prop(group):
    Count = group.Count
    group['Prop'] = Count/Count.sum()
    return group

Nnames = Nnames.groupby(['Year', 'Gender']).apply(add_prop)

Nnames.head()

Below it can be better understood what groupby has done. It has separated the population not only by year, but also by gender, therefore each  girl name's percentage will be dependant of the girls bornt that year.

And all girl's names percentage will ultimately sum 1 (Every year). The same will happen with boys' case.

In [ ]:
Nnames.pivot_table('Prop', index = 'Year', columns = 'Gender', aggfunc = sum).head()

We will now limit our data to the top 1000 names for each sex/year combination.

In [ ]:
def getTop1000(group):
    return group.sort_values(by ="Count", ascending = False)[:1000]

names = Nnames.groupby(['Year', 'Gender']).apply(getTop1000)
names.head()

In [ ]:
names.info()

We split the dataset per gender

In [ ]:
boys = names[names.Gender == "M"]
girls = names[names.Gender =="F"]

We create a dataset that sums the names given each year

In [ ]:
total_born = names.pivot_table('Count', index = 'Year', columns = 'Name', aggfunc=sum)
total_born.head()

We can take a few names, such as Harry, John, Mary and Marylin

In [ ]:
subset = total_born[['Harry', 'John', 'Mary', 'Marylin']] 

In [ ]:
subset.plot(subplots = True, figsize = (12, 10), grid = False, title = 'Number of births per year');

**Measuring the increase in naming diversity**

We will check whether naming diversity has increased or not

In [ ]:
df = names.pivot_table('Prop', index = 'Year', columns = 'Gender', aggfunc = sum)
df.plot();

Now we want to see how many boy names from the top 1000 are needed to compute the 50% of the total in 2010. ('cumsum()' returns the accumulative sum of the value, in this case Proportion)

In [ ]:
df = boys[boys.Year == 2010]
prop_cumsum =df.sort_values(by = 'Prop', ascending = False).Prop.cumsum()
prop_cumsum.head()

We will use the function 'searchsorted' to see where it achieves the 50%, it is after 117 (because array is zero-indexed) different names

In [ ]:
prop_cumsum.searchsorted(0.5)

Nevertheless, in 1900 this number was way smaller (25 names).

In [ ]:
df = boys[boys.Year == 1900]
in1900 = df.sort_values(by = 'Prop', ascending = False).Prop.cumsum()
in1900.searchsorted(0.5)

Following, P4DA, let's extract the last letter from the names, and compare its variance through different ages

In [ ]:
get_letter = lambda x: x[-1]
last_letter = Nnames.Name.map(get_letter)
last_letter.name = 'Last letter'

In [ ]:
table = Nnames.pivot_table('Count', index = last_letter, columns= ['Gender', 'Year'], aggfunc=sum)
table.head()

In [ ]:
subtable = table.reindex(columns = [1910, 1960, 2010], level = 'Year')
subtable.head()


In [ ]:
subtable.sum()

In [ ]:
subtable_prop = subtable/subtable.sum()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


fig, axes = plt.subplots(2, 1, figsize=(10, 8))
subtable_prop['M'].plot(kind='bar', rot=0, ax=axes[0], title='Male')
subtable_prop['F'].plot(kind='bar', rot=0, ax=axes[1], title='Female',
 legend=False);
